# 🔬 DiffLlama vs Llama: Google Colab 实验

这个 Notebook 用于在 Google Colab 环境中运行 DiffLlama 与 Llama 在数学推理任务中的噪声鲁棒性对比实验。

## 📋 实验概述
- **目标**: 对比 DiffLlama-375M 与 Llama-375M 在含噪数学问题上的表现
- **数据集**: GSM8K 数学推理数据集及其噪声变体
- **评估**: 零样本性能 + 注意力机制分析
- **环境**: Google Colab (GPU 推荐)

---

## 🚀 第一步：环境设置

首先检查运行环境并设置必要的配置。

In [ ]:
# 检查 GPU 可用性
import torch
print(f"🖥️  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🔧 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected. Experiment will be slow on CPU.")

🖥️  CUDA available: False
⚠️  No GPU detected. Experiment will be slow on CPU.


In [ ]:
# 如果项目文件不在当前目录，从 Git 仓库克隆
# 替换为您的实际仓库地址
import os
if not os.path.exists('colab/experiment.py'):
    print("📥 Cloning repository...")
    !git clone https://github.com/github-bowen/DiffLlama-Math-Robustness.git
    print("📥 Copying files...")
    !cp -r DiffLlama-Math-Robustness/* .
    print("📥 Removing repository...")
    !rm -rf DiffLlama-Math-Robustness
    print("📥 Done")
else:
    print("✅ Project files found")

✅ Project files found


## 📁 第二步：上传项目文件

如果您没有使用 Git 克隆，请手动上传以下文件到 Colab：

**必需文件**:
- `colab_experiment.py` (主要的 Colab 脚本)
- `pre_download_models.py` (模型下载脚本)
- `src/` 目录下的所有 Python 文件
- `requirements.txt`

使用 Colab 的文件上传功能或从 Google Drive 复制文件。

## 📖 第三步：查看使用说明

运行下面的命令查看详细的使用说明和选项。

In [ ]:
# 显示使用说明
!python colab/experiment.py --instructions


🎯 GOOGLE COLAB USAGE INSTRUCTIONS

1. 📱 Basic Setup (Run once):
   !python colab/experiment.py --setup

2. 🚀 Quick Test (Recommended first run):
   !python colab/experiment.py --mode quick

3. 📊 Medium Experiment:
   !python colab/experiment.py --mode medium --use-drive

4. 🔬 Full Experiment:
   !python colab/experiment.py --mode full --use-drive --max-samples 500

🔧 Options:
   --mode: quick/medium/full
   --use-drive: Save models and results to Google Drive
   --max-samples: Limit number of evaluation samples
   --skip-attention: Skip attention analysis to save time
   --help: Show all options

💡 Tips:
   - Use --use-drive to persist models across sessions
   - Start with quick mode to verify everything works
   - Monitor GPU memory usage in Colab

📁 Results will be saved to:
   - Local: /content/results/
   - Drive: /content/drive/MyDrive/DiffLlama_Experiment/results/



## 🔧 第四步：初始设置

运行初始设置，安装依赖包并配置环境。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 运行初始设置 (包含 Google Drive 挂载)
!python colab/experiment.py --setup --use-drive

🔬 DIFFLAMA VS LLAMA - GOOGLE COLAB EXPERIMENT
🕐 Start time: 2025-06-01 23:13:08
📦 Installing dependencies...
✅ Dependencies installed
🔧 Setting up Colab environment...
🔗 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully
📁 Using Google Drive storage: /content/drive/MyDrive/DiffLlama_Experiment
  ✓ cache -> /content/drive/MyDrive/DiffLlama_Experiment/models
  ✓ data -> /content/drive/MyDrive/DiffLlama_Experiment/data
  ✓ results -> /content/drive/MyDrive/DiffLlama_Experiment/results
✅ Setup completed


In [ ]:
!ls -al

total 100
drwxr-xr-x 1 root root  4096 Jun  1 23:14 .
drwxr-xr-x 1 root root  4096 Jun  1 23:06 ..
lrwxrwxrwx 1 root root    50 Jun  1 23:14 cache -> /content/drive/MyDrive/DiffLlama_Experiment/models
drwxr-xr-x 2 root root  4096 Jun  1 23:14 colab
drwxr-xr-x 4 root root  4096 May 29 14:01 .config
lrwxrwxrwx 1 root root    48 Jun  1 23:14 data -> /content/drive/MyDrive/DiffLlama_Experiment/data
-rw-r--r-- 1 root root 16921 Jun  1 23:07 DiffLlama_Colab_Experiment.ipynb
drwxr-xr-x 2 root root  4096 Jun  1 23:07 docs
drwx------ 6 root root  4096 Jun  1 23:10 drive
-rw-r--r-- 1 root root  2663 Jun  1 23:07 interactive_inference.py
-rw-r--r-- 1 root root  1074 Jun  1 23:07 LICENSE
-rw-r--r-- 1 root root 16092 Jun  1 23:07 main.py
drwxr-xr-x 2 root root  4096 Jun  1 23:11 models_finetuned
-rw-r--r-- 1 root root  8948 Jun  1 23:07 README.md
-rw-r--r-- 1 root root   219 Jun  1 23:13 requirements.txt
lrwxrwxrwx 1 root root    51 Jun  1 23:14 results -> /content/drive/MyDrive/DiffLlama_Experimen

## 🚀 第五步：运行实验

根据您的需求选择合适的实验模式：

### 🏃 快速测试 (推荐首次运行)
使用少量样本验证实验流程，大约需要 30-60 分钟。

In [ ]:
# 快速测试模式
!python colab/experiment.py --mode quick --use-drive

### 📊 中等规模实验
使用中等数量样本，平衡时间和结果质量。

In [ ]:
# 中等规模实验 (运行前请确保快速测试成功)
!python colab/experiment.py --mode medium --use-drive

### 🔬 完整实验
使用完整数据集进行实验，可能需要数小时。

In [ ]:
# 完整实验 (仅在有足够时间时运行)
!python colab/experiment.py --mode full --use-drive --max-samples 500

### 🛠 自定义实验
根据需要调整实验参数。

In [ ]:
# 自定义实验示例
# 只运行评估，跳过注意力分析以节省时间
!python colab/experiment.pyy --mode medium --use-drive --skip-attention --max-samples 100

## 📊 第六步：查看实验结果

实验完成后，查看生成的结果文件。

In [ ]:
# 列出生成的结果文件
!ls -la results/

In [ ]:
# 查看最新的实验摘要
import json
import glob

# 找到最新的摘要文件
summary_files = glob.glob('results/colab_summary_*.json')
if summary_files:
    latest_summary = max(summary_files)
    print(f"📋 Latest experiment summary: {latest_summary}")

    with open(latest_summary, 'r') as f:
        summary = json.load(f)

    print("\n📊 Experiment Summary:")
    for key, value in summary.items():
        print(f"  {key}: {value}")
else:
    print("No experiment summary found. Please run an experiment first.")

In [ ]:
# 显示主要结果
import pandas as pd

# 找到最新的结果文件
result_files = glob.glob('results/colab_results_*.csv')
if result_files:
    latest_results = max(result_files)
    print(f"📈 Latest results: {latest_results}")

    df = pd.read_csv(latest_results)
    print("\n📊 Performance Comparison:")
    print(df.pivot(index='model', columns='dataset', values='accuracy'))

    # 计算性能差异
    pivot_df = df.pivot(index='model', columns='dataset', values='accuracy')
    if 'llama' in pivot_df.index and 'diffllama' in pivot_df.index:
        print("\n🔍 Performance Difference (DiffLlama - Llama):")
        diff = pivot_df.loc['diffllama'] - pivot_df.loc['llama']
        print(diff)
else:
    print("No results found. Please run an experiment first.")

## 📈 第七步：结果可视化

如果实验包含了注意力分析，可以查看生成的注意力热力图。

In [ ]:
# 显示注意力热力图
import matplotlib.pyplot as plt
from IPython.display import Image, display
import os

attention_dir = 'results/attention_maps'
if os.path.exists(attention_dir):
    print("🧠 Attention Visualization Files:")

    # 列出所有注意力图文件
    for root, dirs, files in os.walk(attention_dir):
        for file in files:
            if file.endswith('.png'):
                file_path = os.path.join(root, file)
                print(f"  📊 {file_path}")

                # 显示图片 (可选，取消注释以显示)
                # display(Image(file_path))
else:
    print("No attention maps found. Run experiment with attention analysis enabled.")

In [ ]:
# 显示注意力分析结果
attention_files = glob.glob('results/colab_attention_*.json')
if attention_files:
    latest_attention = max(attention_files)
    print(f"🧠 Latest attention analysis: {latest_attention}")

    with open(latest_attention, 'r') as f:
        attention_data = json.load(f)

    print("\n📊 Attention Allocation Analysis:")
    for model, data in attention_data.items():
        print(f"\n{model.upper()} Model:")
        for condition, stats in data.items():
            print(f"  {condition.capitalize()}:")
            print(f"    KMI (Key Math Info): {stats['kmi_mean']:.3f} ± {stats['kmi_std']:.3f}")
            print(f"    NI (Noise Info): {stats['ni_mean']:.3f} ± {stats['ni_std']:.3f}")
            print(f"    OC (Other Context): {stats['oc_mean']:.3f} ± {stats['oc_std']:.3f}")
else:
    print("No attention analysis found. Run experiment with attention analysis enabled.")

## 💾 第八步：下载结果

将实验结果下载到本地或确保保存在 Google Drive 中。

In [ ]:
# 压缩结果文件以便下载
import zipfile
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_filename = f'experiment_results_{timestamp}.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # 添加结果目录中的所有文件
    for root, dirs, files in os.walk('results'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path)

print(f"📦 Results packaged in: {zip_filename}")
print("You can download this file from Colab's Files panel.")

# 如果使用了 Google Drive，提醒用户
if os.path.exists('/content/drive/MyDrive/DiffLlama_Experiment'):
    print("\n💾 Results are also saved in Google Drive:")
    print("  /content/drive/MyDrive/DiffLlama_Experiment/")

## 🛠 故障排除

如果遇到问题，尝试以下解决方案：

In [ ]:
# 清理 GPU 内存缓存
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ GPU cache cleared")

# 检查可用内存
import psutil
memory = psutil.virtual_memory()
print(f"💾 RAM: {memory.available / 1e9:.1f}GB available / {memory.total / 1e9:.1f}GB total")

In [ ]:
# 如果内存不足，可以重启运行时 (谨慎使用)
# import os
# os.kill(os.getpid(), 9)

## 🎯 实验结论

根据实验结果，您可以分析以下关键问题：

1. **噪声鲁棒性**: DiffLlama 是否在含噪数据上表现更好？
2. **注意力机制**: 差分注意力是否更有效地聚焦关键信息？
3. **性能下降**: 两个模型在不同噪声类型下的性能变化如何？

---

**感谢使用此实验框架！** 🎉

如有问题，请检查：
- GPU 内存是否充足
- 所有必需文件是否上传
- 网络连接是否稳定

**提示**: 建议先运行快速测试模式验证环境，再进行完整实验。